<a href="https://colab.research.google.com/github/springboardmentor891v/Email_Assistant_Using_LangGraph/blob/manoj_preetham/Email_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.9 MB/s eta 0:00:00


In [7]:
# 1. SETUP GMAIL AUTHENTICATION
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']

def get_gmail_service():
    creds = None

    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', SCOPES
        )

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credential.json',
                SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'
            )

            auth_url, _ = flow.authorization_url(
                prompt='consent'
            )

            print("\n🔗 OPEN THIS URL IN YOUR BROWSER:\n")
            print(auth_url)

            code = input("\n📌 Paste the authorization code here: ")
            flow.fetch_token(code=code)
            creds = flow.credentials

        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)

service = get_gmail_service()


🔗 OPEN THIS URL IN YOUR BROWSER:

https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=915502763627-c2ilmm6t0n042r0lbjg7mjr0f0mmqehn.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=CxJ4HBM6mlWprln3EW70gdB0OYNmDD&prompt=consent&access_type=offline

📌 Paste the authorization code here: 4/1ASc3gC3ekad-l0tRj0ncwU0JKSVxeNJMY562jL-gIfCxdka7wDbZbR1vVwY


In [27]:
import base64

def extract_email_data(service, msg_id):
    message = service.users().messages().get(
        userId='me',
        id=msg_id,
        format='full'
    ).execute()

    headers = message['payload']['headers']
    subject = sender = date = ""

    for h in headers:
        if h['name'] == 'Subject':
            subject = h['value']
        elif h['name'] == 'From':
            sender = h['value']
        elif h['name'] == 'Date':
            date = h['value']

    # Extract body
    body = ""
    payload = message['payload']

    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                data = part['body'].get('data')
                if data:
                    body = base64.urlsafe_b64decode(data).decode('utf-8')
    else:
        data = payload['body'].get('data')
        if data:
            body = base64.urlsafe_b64decode(data).decode('utf-8')

    return subject, sender, date, body


# 🔁 Read the emails you listed
for msg in messages:
    msg_id = msg['id']
    subject, sender, date, body = extract_email_data(service, msg_id)

    print("📧 SUBJECT:", subject)
    print("👤 FROM:", sender)
    print("📅 DATE:", date)
    print("📝 BODY (preview):", body[:300])
    print("-" * 60)

📧 SUBJECT: j,hdq
👤 FROM: Chandan <chandankp034@gmail.com>
📅 DATE: Tue, 20 Jan 2026 21:58:12 +0530
📝 BODY (preview): 

------------------------------------------------------------


In [26]:
results = service.users().messages().list(
    userId='me',
    labelIds=['UNREAD'],
    maxResults=1000
).execute()

messages = results.get('messages', [])
print(f"📬 Unread emails found: {len(messages)}")

📬 Unread emails found: 1
